In [5]:
from dotenv import load_dotenv
import google.generativeai as genai
import os
import re

In [9]:
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

In [13]:
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.5-flash")

In [14]:
#codigo de referencia
def extension_a_texto(carpeta):
    """
    Convierte todos los archivos de una carpeta (por ejemplo una extensión de Chrome)
    a un string en el formato esperado por el generador de extensiones.
    """
    bloques = []
    
    for root, _, files in os.walk(carpeta):
        for archivo in files:
            ruta = os.path.join(root, archivo)
            with open(ruta, "r", encoding="utf-8", errors="ignore") as f:
                contenido = f.read()
            
            # Obtenemos la ruta relativa (por si hay subcarpetas)
            ruta_relativa = os.path.relpath(ruta, carpeta)
            
            bloque = f"--- archivo: {ruta_relativa} ---\n{contenido}\n--- fin archivo ---"
            bloques.append(bloque)
    
    return "\n\n".join(bloques)


In [17]:
def generar_extension(prompt_principal, funcionalidades=None, identificadores=None, codigo_referencia=None):
    """
    Genera el código de una extensión de Google Chrome en base a un prompt e información adicional.
    """
    
    mensaje = f"""
    Eres un generador de extensiones de Google Chrome. 
    Debes crear una extensión completamente funcional en base a las siguientes instrucciones.
    
    --- Instrucciones ---
    {prompt_principal.strip()}
    
    --- Funcionalidades requeridas ---
    {funcionalidades if funcionalidades else 'No se especificaron funcionalidades adicionales.'}
    
    --- Identificadores o clases relevantes ---
    {identificadores if identificadores else 'No se especificaron identificadores o clases relevantes.'}
    
    --- Código de referencia ---
    {codigo_referencia if codigo_referencia else 'No se adjuntó código de referencia.'}
    
    --- Formato de salida ---
    Devuelve únicamente el código fuente de la extensión, **sin explicaciones ni texto adicional**,
    en el siguiente formato exacto (respetar guiones y estructura):
    
    --- archivo: manifest.json ---
    (contenido del archivo)
    --- fin archivo ---
    
    --- archivo: background.js ---
    (contenido del archivo)
    --- fin archivo ---
    
    --- archivo: popup.html ---
    (contenido del archivo)
    --- fin archivo ---
    
    Cada archivo debe incluir su contenido completo y funcional.
    No incluyas texto fuera de estos bloques ni encabezados adicionales.
    la extencion no debe poseer referencia a iconos o imagenes.
    Los archivos no deben utilizar o hacer referencia a archivos que no estén incluidos en la respuesta como por ejemplo imagenes.
    """

    respuesta = model.generate_content(mensaje)
    return respuesta.text

In [20]:
prompt = "Crear una extensión que cambie el color de fondo de todas las páginas a modo oscuro con un botón de activación en la barra del navegador."
funcionalidades = "- Botón de encendido/apagado - Persistencia del estado - Inyección de CSS en pestañas activas"
identificadores = None
#codigo_referencia = extension_a_texto("carpeta_extension_referencia")


resultado = generar_extension(prompt, funcionalidades, identificadores)

# Guardamos el resultado en un archivo de texto
with open("extension_generada.txt", "w", encoding="utf-8") as f:
    f.write(resultado)

print("Extensión generada. Revisá el archivo 'extension_generada.txt'.")


Extensión generada. Revisá el archivo 'extension_generada.txt'.


In [21]:
def generar_archivos_extension(codigo_completo, ruta_salida="extension_generada"):
    """
    Genera los archivos de una extensión de Chrome a partir del formato:
    
    --- archivo: nombre ---
    contenido
    --- fin archivo ---
    """
    os.makedirs(ruta_salida, exist_ok=True)

    patron = re.compile(
        r"---\s*archivo:\s*(.*?)\s*---\s*(.*?)\s*---\s*fin\s*archivo\s*---",
        re.DOTALL
    )

    matches = patron.findall(codigo_completo)

    if not matches:
        print("No se encontraron archivos en el texto.")
        return

    for nombre, contenido in matches:
        ruta_archivo = os.path.join(ruta_salida, nombre.strip())
        os.makedirs(os.path.dirname(ruta_archivo), exist_ok=True)
        with open(ruta_archivo, "w", encoding="utf-8") as f:
            f.write(contenido.strip())
        print(f"✅ Archivo generado: {ruta_archivo}")

In [22]:
generar_archivos_extension(resultado)

✅ Archivo generado: extension_generada\manifest.json
✅ Archivo generado: extension_generada\background.js
✅ Archivo generado: extension_generada\popup.html
